In [5]:
library(magrittr)
library(dplyr)

In [3]:
train_set = read.csv("../data/kc_house_train_data.csv")
test_set = read.csv("../data/kc_house_test_data.csv")

In [13]:
ls = lapply(list(train_set, test_set), function(df) {
    mutate(df, 
           bedrooms_squared = bedrooms^2, 
           bed_bath_rooms = bedrooms * bathrooms, 
           log_sqft_living = log(sqft_living), 
           lat_plus_long = lat + long)
})
train_set = ls[[1]]
test_set = ls[[2]]

In [15]:
train_set %>% colnames
train_set %>% colnames %>% length

[1] "id"               "date"             "price"            "bedrooms"        
 [5] "bathrooms"        "sqft_living"      "sqft_lot"         "floors"          
 [9] "waterfront"       "view"             "condition"        "grade"           
[13] "sqft_above"       "sqft_basement"    "yr_built"         "yr_renovated"    
[17] "zipcode"          "lat"              "long"             "sqft_living15"   
[21] "sqft_lot15"       "bedrooms_squared" "bed_bath_rooms"   "log_sqft_living" 
[25] "lat_plus_long"

[1] 25

In [46]:
test_set[22:25] %>% colMeans %>% round(., 2)

bedrooms_squared   bed_bath_rooms  log_sqft_living    lat_plus_long 
           12.45             7.50             7.55           -74.65

In [24]:
mod1 = lm(price ~ sqft_living + bedrooms + bathrooms 
                  + lat + long, train_set)
mod1$coefficients

(Intercept)   sqft_living      bedrooms     bathrooms           lat 
-6.907573e+07  3.122586e+02 -5.958653e+04  1.570674e+04  6.586193e+05 
         long 
-3.093744e+05

In [25]:
mod2 = lm(price ~ sqft_living + bedrooms + bathrooms 
                  + lat + long + bed_bath_rooms, train_set)
mod2$coefficients

(Intercept)    sqft_living       bedrooms      bathrooms            lat 
 -6.686797e+07   3.066101e+02  -1.134464e+05  -7.146131e+04   6.548446e+05 
          long bed_bath_rooms 
 -2.942990e+05   2.557965e+04

In [26]:
mod3 = lm(price ~ sqft_living + bedrooms + bathrooms 
                  + lat + long + bed_bath_rooms + bedrooms_squared 
                  + log_sqft_living + lat_plus_long, train_set)
mod3$coefficients

(Intercept)      sqft_living         bedrooms        bathrooms 
   -6.203608e+07     5.294228e+02     3.451423e+04     6.706078e+04 
             lat             long   bed_bath_rooms bedrooms_squared 
    6.614205e+05    -2.794158e+05    -8.570504e+03    -6.788587e+03 
 log_sqft_living    lat_plus_long 
   -5.618315e+05               NA

In [38]:
sapply(list(mod1, mod2, mod3), function(m) sum(m$residuals^2)) %>% signif(., 2)

[1] 9.7e+14 9.6e+14 9.0e+14

In [45]:
sapply(list(mod1, mod2, mod3), 
       function(m) sum((test_set$price - predict(m, test_set))^2)
       ) %>% signif(., 2)

Warning message:
In predict.lm(m, test_set): prediction from a rank-deficient fit may be misleading

[1] 2.3e+14 2.2e+14 2.6e+14

In [44]:
(test_set$price - predict(mod1, test_set)) %>% head

1           2           3           4           5           6 
 165157.640  195336.996   44856.397    4214.632 -164903.273   41909.298